# 🚀 Complete GPU Server - Image + Voice API
## ✅ Fixes ALL Issues for SDXL-Turbo + Kokoro TTS

### What This Notebook Does:
1. ✅ Sets up SDXL-Turbo for ultra-fast image generation
2. ✅ Sets up Kokoro TTS with proper model initialization
3. ✅ Creates Flask API endpoints for both services
4. ✅ Exposes via ngrok for remote access

### Instructions:
1. Run all cells in order
2. Copy the ngrok URL that appears
3. Paste it into your local app's `.env` file
4. Generate videos with GPU-powered image and voice!

In [ ]:
# 📦 STEP 1: Install All Dependencies
print("\n" + "="*60)
print("📦 Installing Dependencies...")
print("="*60 + "\n")

!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q diffusers transformers accelerate safetensors
!pip install -q flask flask-cors pyngrok
!pip install -q pillow numpy soundfile scipy

# Install Kokoro TTS
!pip install -q kokoro-onnx

print("✅ All dependencies installed!")

In [ ]:
# 🔧 STEP 2: Import Libraries and Setup
print("\n" + "="*60)
print("🔧 Setting up environment...")
print("="*60 + "\n")

import torch
import io
import base64
from PIL import Image
from flask import Flask, request, jsonify, send_file
from flask_cors import CORS
from diffusers import AutoPipelineForText2Image
import numpy as np
import soundfile as sf
from pathlib import Path
import threading
from pyngrok import ngrok

# Check GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🎮 Device: {device}")
if device == "cuda":
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("   ⚠️  No GPU detected - will be slow!")

print("\n✅ Environment ready!")

In [ ]:
# 🎨 STEP 3: Load SDXL-Turbo Model
print("\n" + "="*60)
print("🎨 Loading SDXL-Turbo (Fast Image Generation)...")
print("="*60 + "\n")

# Load SDXL-Turbo pipeline
sdxl_pipe = AutoPipelineForText2Image.from_pretrained(
    "stabilityai/sdxl-turbo",
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    variant="fp16" if device == "cuda" else None
)

sdxl_pipe = sdxl_pipe.to(device)

# Enable memory optimizations
if device == "cuda":
    sdxl_pipe.enable_xformers_memory_efficient_attention()
    print("   ✅ Memory optimizations enabled")

print("\n✅ SDXL-Turbo loaded and ready!")
print(f"   Model: stabilityai/sdxl-turbo")
print(f"   Device: {device}")
print(f"   Speed: 1-4 steps (ultra-fast!)")

In [ ]:
# 🎤 STEP 4: Setup Kokoro TTS
print("\n" + "="*60)
print("🎤 Setting up Kokoro TTS (Professional Voice)...")
print("="*60 + "\n")

from kokoro import Kokoro

# Download and cache Kokoro models
print("📥 Downloading Kokoro models (first time only)...")

# Create Kokoro instance with automatic model download
kokoro = None

def get_kokoro_instance():
    """Lazy load Kokoro instance"""
    global kokoro
    if kokoro is None:
        print("   Loading Kokoro TTS models...")
        # Initialize Kokoro with default models (will auto-download)
        kokoro = Kokoro.from_pretrained()
        print("   ✅ Kokoro loaded!")
    return kokoro

# Voice mapping from user-friendly names to Kokoro voices
VOICE_MAP = {
    'aria': 'af_sarah',  # Professional female
    'guy': 'am_adam',    # Professional male
    'jenny': 'af_nicole', # Cheerful female
    'christopher': 'am_michael', # Casual male
    'sara': 'af_bella',  # Young female
    'roger': 'am_adam',  # Authoritative male
    'nancy': 'af_jessica', # Professional female
    'andrew': 'am_adam', # Business male
}

print("\n✅ Kokoro TTS ready!")
print(f"   Voices: {len(VOICE_MAP)} available")
print(f"   Quality: Professional studio quality")

In [ ]:
# 🔌 STEP 5: Create Flask API Server
print("\n" + "="*60)
print("🔌 Creating Flask API server...")
print("="*60 + "\n")

app = Flask(__name__)
CORS(app)

# ═══════════════════════════════════════════════════════════════
# IMAGE GENERATION ENDPOINT
# ═══════════════════════════════════════════════════════════════

@app.route('/generate_image', methods=['POST'])
def generate_image():
    """Generate image with SDXL-Turbo"""
    try:
        data = request.json
        prompt = data.get('prompt', '')
        
        if not prompt:
            return jsonify({'error': 'Prompt is required'}), 400
        
        print(f"\n🎨 Generating image...")
        print(f"   Prompt: {prompt[:100]}...")
        
        # Generate image with SDXL-Turbo (1-4 steps for speed!)
        image = sdxl_pipe(
            prompt=prompt,
            num_inference_steps=2,  # Ultra-fast!
            guidance_scale=0.0,     # Turbo doesn't need guidance
            height=1024,
            width=1024
        ).images[0]
        
        # Convert to bytes
        img_byte_arr = io.BytesIO()
        image.save(img_byte_arr, format='PNG')
        img_byte_arr.seek(0)
        
        # Convert to base64
        img_base64 = base64.b64encode(img_byte_arr.getvalue()).decode('utf-8')
        
        print(f"   ✅ Image generated successfully!")
        
        return jsonify({
            'success': True,
            'image': img_base64,
            'format': 'png'
        })
    
    except Exception as e:
        print(f"   ❌ Error: {str(e)}")
        return jsonify({'error': str(e)}), 500

# ═══════════════════════════════════════════════════════════════
# VOICE GENERATION ENDPOINT
# ═══════════════════════════════════════════════════════════════

@app.route('/generate_audio', methods=['POST'])
def generate_audio():
    """Generate audio with Kokoro TTS"""
    try:
        data = request.json
        text = data.get('text', '')
        voice = data.get('voice', 'aria')
        speed = float(data.get('speed', 1.0))
        
        if not text:
            return jsonify({'error': 'Text is required'}), 400
        
        # Map voice name to Kokoro voice
        kokoro_voice = VOICE_MAP.get(voice.lower(), 'af_sarah')
        
        print(f"\n🎤 Generating audio...")
        print(f"   Voice: {voice} → {kokoro_voice}")
        print(f"   Speed: {speed}x")
        print(f"   Text length: {len(text)} characters")
        
        # Get Kokoro instance
        k = get_kokoro_instance()
        
        # Generate audio
        # Split long text into chunks for better processing
        if len(text) > 1000:
            print(f"   📝 Splitting long text into chunks...")
            chunks = split_text_smart(text, max_chars=1000)
            audio_chunks = []
            
            for i, chunk in enumerate(chunks):
                print(f"   Processing chunk {i+1}/{len(chunks)}...")
                audio_chunk = k.generate(chunk, voice=kokoro_voice, speed=speed)
                audio_chunks.append(audio_chunk)
            
            # Concatenate all chunks
            audio = np.concatenate(audio_chunks)
        else:
            audio = k.generate(text, voice=kokoro_voice, speed=speed)
        
        # Save to bytes
        audio_byte_arr = io.BytesIO()
        sf.write(audio_byte_arr, audio, k.sample_rate, format='WAV')
        audio_byte_arr.seek(0)
        
        # Convert to base64
        audio_base64 = base64.b64encode(audio_byte_arr.getvalue()).decode('utf-8')
        
        duration = len(audio) / k.sample_rate
        print(f"   ✅ Audio generated! Duration: {duration:.1f}s")
        
        return jsonify({
            'success': True,
            'audio': audio_base64,
            'format': 'wav',
            'duration': duration,
            'sample_rate': k.sample_rate
        })
    
    except Exception as e:
        print(f"   ❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()
        return jsonify({'error': str(e)}), 500

# ═══════════════════════════════════════════════════════════════
# HEALTH CHECK
# ═══════════════════════════════════════════════════════════════

@app.route('/health', methods=['GET'])
def health():
    """Health check endpoint"""
    return jsonify({
        'status': 'ok',
        'device': device,
        'services': {
            'sdxl_turbo': 'ready',
            'kokoro_tts': 'ready'
        }
    })

# ═══════════════════════════════════════════════════════════════
# HELPER FUNCTIONS
# ═══════════════════════════════════════════════════════════════

def split_text_smart(text, max_chars=1000):
    """Split text at sentence boundaries"""
    sentences = text.replace('!', '.').replace('?', '.').split('.')
    sentences = [s.strip() + '.' for s in sentences if s.strip()]
    
    chunks = []
    current_chunk = ""
    
    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= max_chars:
            current_chunk += " " + sentence
        else:
            if current_chunk:
                chunks.append(current_chunk.strip())
            current_chunk = sentence
    
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    return chunks if chunks else [text]

print("\n✅ Flask API server created!")
print("   Endpoints:")
print("   POST /generate_image - Generate images with SDXL-Turbo")
print("   POST /generate_audio - Generate audio with Kokoro TTS")
print("   GET  /health - Health check")

In [ ]:
# 🌐 STEP 6: Setup ngrok Tunnel
print("\n" + "="*60)
print("🌐 Setting up ngrok tunnel...")
print("="*60 + "\n")

# Set your ngrok auth token here (get from: https://dashboard.ngrok.com/get-started/your-authtoken)
NGROK_AUTH_TOKEN = "YOUR_NGROK_TOKEN_HERE"  # Replace with your token

# If you don't have a token, get one free at: https://ngrok.com/
if NGROK_AUTH_TOKEN == "YOUR_NGROK_TOKEN_HERE":
    print("⚠️  Please set your ngrok auth token!")
    print("   1. Go to: https://dashboard.ngrok.com/get-started/your-authtoken")
    print("   2. Copy your token")
    print("   3. Replace NGROK_AUTH_TOKEN in the cell above")
    print("   4. Run this cell again")
else:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    print("✅ ngrok token set!")

print("\n📝 Note: If you don't have an ngrok token:")
print("   - Free tier: 1 concurrent tunnel, no custom domains")
print("   - Sign up at: https://ngrok.com/")

In [ ]:
# 🚀 STEP 7: Start Server!
print("\n" + "="*60)
print("🚀 STARTING GPU SERVER!")
print("="*60 + "\n")

# Start ngrok tunnel
public_url = ngrok.connect(5000, bind_tls=True)

print("\n" + "="*60)
print("✅ GPU SERVER IS RUNNING!")
print("="*60)
print(f"\n🌐 Your Public URL:")
print(f"   {public_url}")
print("\n📋 Copy this URL and use it in your local app!")
print("\n🔧 Setup Instructions:")
print("   1. Copy the URL above")
print("   2. In your local project, create/update .env file:")
print(f"      SDXL_TURBO_API_URL={public_url}/generate_image")
print(f"      KOKORO_API_URL={public_url}/generate_audio")
print("   3. Restart your local app")
print("   4. Generate videos with GPU power!")
print("\n💡 Test endpoints:")
print(f"   Health: {public_url}/health")
print(f"   Images: {public_url}/generate_image (POST)")
print(f"   Audio:  {public_url}/generate_audio (POST)")
print("\n⚠️  Keep this notebook running! Closing it will stop the server.")
print("="*60 + "\n")

# Start Flask server (this will block - server runs forever)
app.run(port=5000, debug=False)